# Step 3: Model Operationalization & Deployment

In the previous script, you learned how to save lstm trained models to files. You also learned that model weights are easily stored using  HDF5 format and that the network structure can be saved in JSON. In this script, you will learn how to load your models up, operationalize and use them to make future predictions.

In [128]:
import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import urllib
import json
import shutil
from keras.models import model_from_json

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM, Activation gdt

import h5py
from sklearn import datasets

# Setup the pyspark environment
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
spark = SparkSession.builder.getOrCreate()


# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

In [97]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<your blob storage account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
CONTAINER_NAME = "sensordataingestionpm"

# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# We will store and read each of these data sets in blob storage in an 
# Azure Storage Container on your Azure subscription.
# See https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
# for details.

# This is the final feature data file.
TEST_DATA = 'PM_test_files.parquet'

# This is where we store the final model data file.
LOCAL_DIRECT = 'model_result.parquet'

In [98]:
# load the previous created final dataset into the workspace
# create a local path where we store results
if not os.path.exists(TEST_DATA):
    os.makedirs(TEST_DATA)
    print('DONE creating a local directory!')

# download the entire parquet result folder to local path for a new run 
for blob in az_blob_service.list_blobs(CONTAINER_NAME):
    if TRAIN_DATA in blob.name:
        local_file = os.path.join(TEST_DATA, os.path.basename(blob.name))
        az_blob_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)
        
test_df = spark.read.parquet(TEST_DATA)
test_df = test_df.toPandas()
test_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,86,69,0.591954,0.333333,0.0,0.0,0.439759,0.450621,0.380655,0.0,...,0.0,0.500000,0.0,0.0,0.465116,0.567109,0.188366,209,0,0
1,86,70,0.356322,0.333333,0.0,0.0,0.472892,0.496621,0.293889,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.536868,0.191136,208,0,0
2,86,71,0.609195,0.500000,0.0,0.0,0.496988,0.348812,0.447670,0.0,...,0.0,0.416667,0.0,0.0,0.457364,0.624827,0.193906,207,0,0
3,86,72,0.454023,0.666667,0.0,0.0,0.289157,0.306736,0.451047,0.0,...,0.0,0.416667,0.0,0.0,0.550388,0.579674,0.196676,206,0,0
4,86,73,0.620690,0.166667,0.0,0.0,0.409639,0.418574,0.362762,0.0,...,0.0,0.250000,0.0,0.0,0.612403,0.615576,0.199446,205,0,0
5,86,74,0.666667,0.833333,0.0,0.0,0.484940,0.296926,0.294227,0.0,...,0.0,0.500000,0.0,0.0,0.651163,0.579950,0.202216,204,0,0
6,86,75,0.620690,0.833333,0.0,0.0,0.496988,0.410726,0.447164,0.0,...,0.0,0.416667,0.0,0.0,0.573643,0.519470,0.204986,203,0,0
7,86,76,0.511494,0.416667,0.0,0.0,0.427711,0.301068,0.392134,0.0,...,0.0,0.500000,0.0,0.0,0.604651,0.671500,0.207756,202,0,0
8,86,77,0.442529,0.583333,0.0,0.0,0.364458,0.285372,0.364450,0.0,...,0.0,0.333333,0.0,0.0,0.527132,0.784176,0.210526,201,0,0
9,86,78,0.459770,0.583333,0.0,0.0,0.433735,0.558535,0.465901,0.0,...,0.0,0.416667,0.0,0.0,0.534884,0.593897,0.213296,200,0,0


In [99]:
with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.json', 'r') as json_file:
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("json file read from shared folder")

json file read from shared folder


In [100]:
with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.h5', 'r') as model:
    loaded_model.load_weights(os.path.join(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'], 'modellstm.h5'))
    print("model weights read from shared folder")

model weights read from shared folder


### Test init() and run() functions to read from the working directory

In [101]:
# pick a large window size of 50 cycles
sequence_length = 50

In [118]:
def init():
    # read in the model file
    from keras.models import model_from_json

    # load json and create model
    with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.json', 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.h5', 'r') as model:
        print("Loaded model")
    
    #inputs_dc = ModelDataCollector("modellstm.h5", identifier="inputs")
    #print(inputs_dc)
    #prediction_dc = ModelDataCollector("modellstm.h5", identifier="prediction")


In [119]:
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [120]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
input_features = sensor_cols + sequence_cols
input_features
sequence_cols.extend(sensor_cols)

In [121]:
# generator for the sequences
seq_gen = (list(gen_sequence(test_df[test_df['id']==id], sequence_length, sequence_cols)) 
           for id in test_df['id'].unique())

In [122]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(28727, 50, 25)

In [123]:
def run(seq_array): 
    global clf2, inputs_dc, prediction_dc
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        #inputs_dc.collect(seq_array)
        #prediction_dc.collect(prediction)
        return (prediction)
    except Exception as e:
        return(str(e))

In [124]:
prediction = loaded_model.predict_proba(seq_array)
prediction

array([[ 0.00263677],
       [ 0.0027143 ],
       [ 0.00279908],
       ..., 
       [ 0.00163748],
       [ 0.00169866],
       [ 0.00178284]], dtype=float32)

In [125]:
init()

Loaded model


In [126]:
run(seq_array)

array([[ 0.00263677],
       [ 0.0027143 ],
       [ 0.00279908],
       ..., 
       [ 0.00163748],
       [ 0.00169866],
       [ 0.00178284]], dtype=float32)

## Persist model assets

Next we persist the assets we have created to disk for use in operationalization.

In [127]:
%%writefile {os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']}/lstmscore.py

import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import urllib
import json
import shutil

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation

def init():
    # read in the model file
    from keras.models import model_from_json

    # load json and create model
    json_file = open('modellstm.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("modellstm.h5")
    print("Loaded model")
    
    #inputs_dc = ModelDataCollector("modellstm.h5", identifier="inputs")
    #print(inputs_dc)
    #prediction_dc = ModelDataCollector("modellstm.h5", identifier="prediction")
    
    
def run(seq_array): 
    global clf2, inputs_dc, prediction_dc
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        #inputs_dc.collect(seq_array)
        #prediction_dc.collect(prediction)
        return (prediction)
    except Exception as e:
        return(str(e))

Overwriting /azureml-share//lstmscore.py


Retrieving the files from the shared folder:
https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

## Deployment

Once the assets are stored, we can download them into a local compute context for operationalization on an Azure web service.

We demonstrate how to setup this web service this through a CLI window opened in the AML Workbench application. 

## Download the model

To download the model we've saved, follow these instructions on a local computer.

- Open the [Azure Portal](http://portal.azure.com)
- In the left hand pane, click on __All resources__
- Search for the storage account using the name you provided earlier in this notebook. 
- Choose the storage account from search result list, this will open the storage account panel.
- On the storage account panel, choose __Blobs__
- On the Blobs panel choose the container __modeldeploy__
- Select the file o16n.zip and on the properties pane for that blob choose download.

Once downloaded, unzip the file into the directory of your choosing. The zip file contains three deployment assets:

- the `lstmscore.py` file
- a `lstm.model` directory
- the `service_schema.json` file



## Create a model management endpoint 

Create a modelmanagement under your account. We will call this `lstmmodelmanagement`. The remaining defaults are acceptable.

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`


## Check environment settings

Show what environment is currently active:

`az ml env show`

If nothing is set, we setup the environment with the existing model management context first: 

` az ml env setup --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

then set the current environment:

`az ml env set --resource-group <RESOURCE_GROUP> --cluster-name pdmmodelmanagement`

Check that the environment is now set:

`az ml env show`


## Deploy your web service 

Once the environment is setup, we'll deploy the web service from the CLI.

These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (`lstmscore.py`, `service_schema.json` and `lstm.model`). If your kernel has run locally, the assets will be in the `os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']`. 

On windows this points to:

```
cd C:\Users\<username>\.azureml\share\<team account>\<Project Name>
```

on linux variants this points to:

```
cd ~\.azureml\share\<team account>\<Project Name>
```


The command to create a web service (`<SERVICE_ID>`) with these operationalization assets in the current directory is:

```
az ml service create realtime -f <filename> -r <TARGET_RUNTIME> -m <MODEL_FILE> -s <SCHEMA_FILE> -n <SERVICE_ID> --cpu 0.1
```

The default cluster has only 2 nodes with 2 cores each. Some cores are taken for system components. AMLWorkbench asks for 1 core per service. To deploy multiple services into this cluster, we specify the cpu requirement in the service create command as (--cpu 0.1) to request 10% of a core. 

For this example, we will call our webservice `amlworkbenchpdmwebservice`. This `SERVICE_ID` must be all lowercase, with no spaces:

```
az ml service create realtime -f lstmscore.py -r spark-py -m lstm.model -s service_schema.json --cpu 0.1 -n amlworkbenchpdmwebservice
```

This command will take some time to execute. 

Once complete, the command returns sample usage commands to test the service for both PowerShell and the cmd prompt. We can execute these commands from the command line as well.